Reference
- https://www.kaggle.com/ymlai87416/web-traffic-time-series-forecast-with-4-model
- https://github.com/jfpuget/Kaggle (this boilplate)

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/kaggle-web-traffic-time-series-forecasting/codes

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/kaggle-web-traffic-time-series-forecasting/codes


In [0]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 600
from tqdm import tqdm
import gc

from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.decomposition import PCA

from keras.layers.normalization import BatchNormalization

from keras.models import Sequential, Model

from keras.layers import Input, Embedding, Dense, Activation, Dropout, Flatten

from keras import regularizers 

import keras

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GroupKFold


Using TensorFlow backend.


In [0]:
def init():
    np.random.seed = 0
    
init()

In [0]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

def smape2D(y_true, y_pred):
    return smape(np.ravel(y_true), np.ravel(y_pred))
    
def smape_mask(y_true, y_pred, threshold):
    denominator = (np.abs(y_true) + np.abs(y_pred)) 
    diff = np.abs(y_true - y_pred) 
    diff[denominator == 0] = 0.0
    
    return diff <= (threshold / 2.0) * denominator

## Basic information of the dataset

### Dataset
- 145K wiki page
- Training data from 2015-07-01 to 2017-09-10
- Testing data from 2017-09-13 to 2017-11-13


In [0]:
orig = pd.read_csv("../data/train_2.csv")
orig.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,...,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-

## Prepare training and testing dataset and data cleaning

###Loading training dataset

In [0]:
max_size = 366 # number of days in 2015 with 3 days before end
offset = 0.5

all_page = orig.Page.copy()
train_key = orig[['Page']].copy()
train_all = orig.copy()
train_all = train_all.iloc[:,1:] * offset
train_all.head()

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-19,2015-09-20,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-26,2015-09-27,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-03,2015-10-04,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-10,2015-10-11,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-17,2015-10-18,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-24,2015-10-25,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-07,2015-11-08,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-21,2015-11-22,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-28,2015-11-29,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-05,2015-12-06,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-12,2015-12-13,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-19,2015-12-20,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-26,2015-12-27,2015-12-28,2015-12-29,2015-12-30,2015-12-31,2016-01-01,2016-01-02,2016-01-03,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-09,2016-01-10,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,2016-01-16,2016-01-17,2016-01-18,2016-01-19,2016-01-20,2016-01-21,2016-01-22,2016-01-23,2016-01-24,2016-01-25,2016-01-26,2016-01-27,2016-01-28,2016-01-29,2016-01-30,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,2016-02-11,2016-02-12,2016-02-13,2016-02-14,2016-02-15,2016-02-16,2016-02-17,2016-02-18,2016-02-19,2016-02-20,2016-02-21,2016-02-22,2016-02-23,2016-02-24,2016-02-25,2016-02-26,2016-02-27,2016-02-28,2016-02-29,2016-03-01,2016-03-02,2016-03-03,2016-03-04,2016-03-05,2016-03-06,2016-03-07,2016-03-08,2016-03-09,2016-03-10,2016-03-11,2016-03-12,2016-03-13,2016-03-14,2016-03-15,2016-03-16,2016-03-17,2016-03-18,2016-03-19,2016-03-20,2016-03-21,2016-03-22,2016-03-23,2016-03-24,2016-03-25,2016-03-26,2016-03-27,2016-03-28,2016-03-29,2016-03-30,2016-03-31,2016-04-01,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24,2016-04-25,...,2016-11-15,2016-11-16,2016-11-17,2016-11-18,2016-11-19,2016-11-20,2016-11-21,2016-11-22,2016-11-23,2016-11-24,2016-11-25,2016-11-26,2016-11-27,2016-11-28,2016-11-29,2016-11-30,2016-12-01,2016-12-02,2016-12-03,2016-12-04,2016-12-05,2016-12-06,2016-12-07,2016-12-08,2016-12-09,2016-12-10,2016-12-11,2016-12-12,2016-12-13,2016-12-14,2016-12-15,2016-12-16,2016-12-17,2016-12-18,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,20

In [0]:
def get_date_index(date, train_all=train_all):
    for idx, c in enumerate(train_all.columns):
        if date == c:
            break
    if idx == len(train_all.columns):
        return None
    return idx

### Split training dataset into training and testing
Train data is extract from the original data by the last 181 days

In [0]:
# Split the dataset into training and testing datasets
train_end = get_date_index('2017-09-10') + 1
test_start = get_date_index('2017-09-12')

train = train_all.iloc[ : , (train_end - max_size) : train_end].copy().astype('float32')
train = train.iloc[:,::-1].copy().astype('float32')

In [0]:
print(train.shape)
train.head()

(145063, 366)


,2017-09-10,2017-09-09,2017-09-08,2017-09-07,2017-09-06,2017-09-05,2017-09-04,2017-09-03,2017-09-02,2017-09-01,2017-08-31,2017-08-30,2017-08-29,2017-08-28,2017-08-27,2017-08-26,2017-08-25,2017-08-24,2017-08-23,2017-08-22,2017-08-21,2017-08-20,2017-08-19,2017-08-18,2017-08-17,2017-08-16,2017-08-15,2017-08-14,2017-08-13,2017-08-12,2017-08-11,2017-08-10,2017-08-09,2017-08-08,2017-08-07,2017-08-06,2017-08-05,2017-08-04,2017-08-03,2017-08-02,2017-08-01,2017-07-31,2017-07-30,2017-07-29,2017-07-28,2017-07-27,2017-07-26,2017-07-25,2017-07-24,2017-07-23,2017-07-22,2017-07-21,2017-07-20,2017-07-19,2017-07-18,2017-07-17,2017-07-16,2017-07-15,2017-07-14,2017-07-13,2017-07-12,2017-07-11,2017-07-10,2017-07-09,2017-07-08,2017-07-07,2017-07-06,2017-07-05,2017-07-04,2017-07-03,2017-07-02,2017-07-01,2017-06-30,2017-06-29,2017-06-28,2017-06-27,2017-06-26,2017-06-25,2017-06-24,2017-06-23,2017-06-22,2017-06-21,2017-06-20,2017-06-19,2017-06-18,2017-06-17,2017-06-16,2017-06-15,2017-06-14,2017-06-13,2017-06-12,2017-06-11,2017-06-10,2017-06-09,2017-06-08,2017-06-07,2017-06-06,2017-06-05,2017-06-04,2017-06-03,2017-06-02,2017-06-01,2017-05-31,2017-05-30,2017-05-29,2017-05-28,2017-05-27,2017-05-26,2017-05-25,2017-05-24,2017-05-23,2017-05-22,2017-05-21,2017-05-20,2017-05-19,2017-05-18,2017-05-17,2017-05-16,2017-05-15,2017-05-14,2017-05-13,2017-05-12,2017-05-11,2017-05-10,2017-05-09,2017-05-08,2017-05-07,2017-05-06,2017-05-05,2017-05-04,2017-05-03,2017-05-02,2017-05-01,2017-04-30,2017-04-29,2017-04-28,2017-04-27,2017-04-26,2017-04-25,2017-04-24,2017-04-23,2017-04-22,2017-04-21,2017-04-20,2017-04-19,2017-04-18,2017-04-17,2017-04-16,2017-04-15,2017-04-14,2017-04-13,2017-04-12,2017-04-11,2017-04-10,2017-04-09,2017-04-08,2017-04-07,2017-04-06,2017-04-05,2017-04-04,2017-04-03,2017-04-02,2017-04-01,2017-03-31,2017-03-30,2017-03-29,2017-03-28,2017-03-27,2017-03-26,2017-03-25,2017-03-24,2017-03-23,2017-03-22,2017-03-21,2017-03-20,2017-03-19,2017-03-18,2017-03-17,2017-03-16,2017-03-15,2017-03-14,2017-03-13,2017-03-12,2017-03-11,2017-03-10,2017-03-09,2017-03-08,2017-03-07,2017-03-06,2017-03-05,2017-03-04,2017-03-03,2017-03-02,2017-03-01,2017-02-28,2017-02-27,2017-02-26,2017-02-25,2017-02-24,2017-02-23,2017-02-22,2017-02-21,2017-02-20,2017-02-19,2017-02-18,2017-02-17,2017-02-16,2017-02-15,2017-02-14,2017-02-13,2017-02-12,2017-02-11,2017-02-10,2017-02-09,2017-02-08,2017-02-07,2017-02-06,2017-02-05,2017-02-04,2017-02-03,2017-02-02,2017-02-01,2017-01-31,2017-01-30,2017-01-29,2017-01-28,2017-01-27,2017-01-26,2017-01-25,2017-01-24,2017-01-23,2017-01-22,2017-01-21,2017-01-20,2017-01-19,2017-01-18,2017-01-17,2017-01-16,2017-01-15,2017-01-14,2017-01-13,2017-01-12,2017-01-11,2017-01-10,2017-01-09,2017-01-08,2017-01-07,2017-01-06,2017-01-05,2017-01-04,2017-01-03,2017-01-02,2017-01-01,2016-12-31,2016-12-30,2016-12-29,2016-12-28,2016-12-27,2016-12-26,2016-12-25,2016-12-24,2016-12-23,2016-12-22,2016-12-21,2016-12-20,2016-12-19,2016-12-18,2016-12-17,2016-12-16,2016-12-15,2016-12-14,2016-12-13,2016-12-12,2016-12-11,2016-12-10,2016-12-09,2016-12-08,2016-12-07,2016-12-06,2016-12-05,2016-12-04,2016-12-03,2016-12-02,2016-12-01,2016-11-30,2016-11-29,2016-11-28,2016-11-27,2016-11-26,2016-11-25,2016-11-24,2016-11-23,2016-11-22,2016-11-21,2016-11-20,2016-11-19,2016-11-18,2016-11-17,2016-11-16,2016-11-15,2016-11-14,2016-11-13,2016-11-12,2016-11-11,2016-11-10,2016-11-09,2016-11-08,2016-11-07,2016-11-06,2016-11-05,2016-11-04,2016-11-03,2016-11-02,2016-11-01,2016-10-31,2016-10-30,2016-10-29,2016-10-28,2016-10-27,2016-10-26,2016-10-25,2016-10-24,2016-10-23,2016-10-22,2016-10-21,2016-10-20,2016-10-19,2016-10-18,2016-10-17,2016-10-16,2016-10-15,2016-10-14,2016-10-13,2016-10-12,2016-10-11,2016-10-10,2016-10-09,2016-10-08,2016-10-07,2016-10-06,2016-10-05,2016-10-04,2016-10-03,2016-10-02,2016-10-01,2016-09-30,2016-09-29,2016-09-28,2016-09-27,2016-09-26,2016-09-25,2016-09-24,2016-09-23,2016-09-22,2016-09-21,2016-09-20,2016-09-19,2016-09-18,2016-09-17,2016-09-16,2016-09-15,2016-09-14,2016-09-13,2016-0

### Feature Engineering

In [0]:
data = [page.split('_') for page in tqdm(train_key.Page)]

access = ['_'.join(page[-2:]) for page in data]

site = [page[-3][:2] for page in data]

page = ['_'.join(page[:-3]) for page in data]
page[:2]

train_key['PageTitle'] = page
train_key['Site'] = site
train_key['AccessAgent'] = access
train_key.head()

100%|██████████| 145063/145063 [00:00<00:00, 339067.12it/s]


,Page,PageTitle,Site,AccessAgent
0,2NE1_zh.wikipedia.org_all-access_spider,2NE1,zh,all-access_spider
1,2PM_zh.wikipedia.org_all-access_spider,2PM,zh,all-access_spider
2,3C_zh.wikipedia.org_all-access_spider,3C,zh,all-access_spider
3,4minute_zh.wikipedia.org_all-access_spider,4minute,zh,all-access_spider
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You,zh,all-access_spider


In [0]:
train_norm = np.log1p(train).astype('float32')
train_norm.head()

,2017-09-10,2017-09-09,2017-09-08,2017-09-07,2017-09-06,2017-09-05,2017-09-04,2017-09-03,2017-09-02,2017-09-01,2017-08-31,2017-08-30,2017-08-29,2017-08-28,2017-08-27,2017-08-26,2017-08-25,2017-08-24,2017-08-23,2017-08-22,2017-08-21,2017-08-20,2017-08-19,2017-08-18,2017-08-17,2017-08-16,2017-08-15,2017-08-14,2017-08-13,2017-08-12,2017-08-11,2017-08-10,2017-08-09,2017-08-08,2017-08-07,2017-08-06,2017-08-05,2017-08-04,2017-08-03,2017-08-02,2017-08-01,2017-07-31,2017-07-30,2017-07-29,2017-07-28,2017-07-27,2017-07-26,2017-07-25,2017-07-24,2017-07-23,2017-07-22,2017-07-21,2017-07-20,2017-07-19,2017-07-18,2017-07-17,2017-07-16,2017-07-15,2017-07-14,2017-07-13,2017-07-12,2017-07-11,2017-07-10,2017-07-09,2017-07-08,2017-07-07,2017-07-06,2017-07-05,2017-07-04,2017-07-03,2017-07-02,2017-07-01,2017-06-30,2017-06-29,2017-06-28,2017-06-27,2017-06-26,2017-06-25,2017-06-24,2017-06-23,2017-06-22,2017-06-21,2017-06-20,2017-06-19,2017-06-18,2017-06-17,2017-06-16,2017-06-15,2017-06-14,2017-06-13,2017-06-12,2017-06-11,2017-06-10,2017-06-09,2017-06-08,2017-06-07,2017-06-06,2017-06-05,2017-06-04,2017-06-03,2017-06-02,2017-06-01,2017-05-31,2017-05-30,2017-05-29,2017-05-28,2017-05-27,2017-05-26,2017-05-25,2017-05-24,2017-05-23,2017-05-22,2017-05-21,2017-05-20,2017-05-19,2017-05-18,2017-05-17,2017-05-16,2017-05-15,2017-05-14,2017-05-13,2017-05-12,2017-05-11,2017-05-10,2017-05-09,2017-05-08,2017-05-07,2017-05-06,2017-05-05,2017-05-04,2017-05-03,2017-05-02,2017-05-01,2017-04-30,2017-04-29,2017-04-28,2017-04-27,2017-04-26,2017-04-25,2017-04-24,2017-04-23,2017-04-22,2017-04-21,2017-04-20,2017-04-19,2017-04-18,2017-04-17,2017-04-16,2017-04-15,2017-04-14,2017-04-13,2017-04-12,2017-04-11,2017-04-10,2017-04-09,2017-04-08,2017-04-07,2017-04-06,2017-04-05,2017-04-04,2017-04-03,2017-04-02,2017-04-01,2017-03-31,2017-03-30,2017-03-29,2017-03-28,2017-03-27,2017-03-26,2017-03-25,2017-03-24,2017-03-23,2017-03-22,2017-03-21,2017-03-20,2017-03-19,2017-03-18,2017-03-17,2017-03-16,2017-03-15,2017-03-14,2017-03-13,2017-03-12,2017-03-11,2017-03-10,2017-03-09,2017-03-08,2017-03-07,2017-03-06,2017-03-05,2017-03-04,2017-03-03,2017-03-02,2017-03-01,2017-02-28,2017-02-27,2017-02-26,2017-02-25,2017-02-24,2017-02-23,2017-02-22,2017-02-21,2017-02-20,2017-02-19,2017-02-18,2017-02-17,2017-02-16,2017-02-15,2017-02-14,2017-02-13,2017-02-12,2017-02-11,2017-02-10,2017-02-09,2017-02-08,2017-02-07,2017-02-06,2017-02-05,2017-02-04,2017-02-03,2017-02-02,2017-02-01,2017-01-31,2017-01-30,2017-01-29,2017-01-28,2017-01-27,2017-01-26,2017-01-25,2017-01-24,2017-01-23,2017-01-22,2017-01-21,2017-01-20,2017-01-19,2017-01-18,2017-01-17,2017-01-16,2017-01-15,2017-01-14,2017-01-13,2017-01-12,2017-01-11,2017-01-10,2017-01-09,2017-01-08,2017-01-07,2017-01-06,2017-01-05,2017-01-04,2017-01-03,2017-01-02,2017-01-01,2016-12-31,2016-12-30,2016-12-29,2016-12-28,2016-12-27,2016-12-26,2016-12-25,2016-12-24,2016-12-23,2016-12-22,2016-12-21,2016-12-20,2016-12-19,2016-12-18,2016-12-17,2016-12-16,2016-12-15,2016-12-14,2016-12-13,2016-12-12,2016-12-11,2016-12-10,2016-12-09,2016-12-08,2016-12-07,2016-12-06,2016-12-05,2016-12-04,2016-12-03,2016-12-02,2016-12-01,2016-11-30,2016-11-29,2016-11-28,2016-11-27,2016-11-26,2016-11-25,2016-11-24,2016-11-23,2016-11-22,2016-11-21,2016-11-20,2016-11-19,2016-11-18,2016-11-17,2016-11-16,2016-11-15,2016-11-14,2016-11-13,2016-11-12,2016-11-11,2016-11-10,2016-11-09,2016-11-08,2016-11-07,2016-11-06,2016-11-05,2016-11-04,2016-11-03,2016-11-02,2016-11-01,2016-10-31,2016-10-30,2016-10-29,2016-10-28,2016-10-27,2016-10-26,2016-10-25,2016-10-24,2016-10-23,2016-10-22,2016-10-21,2016-10-20,2016-10-19,2016-10-18,2016-10-17,2016-10-16,2016-10-15,2016-10-14,2016-10-13,2016-10-12,2016-10-11,2016-10-10,2016-10-09,2016-10-08,2016-10-07,2016-10-06,2016-10-05,2016-10-04,2016-10-03,2016-10-02,2016-10-01,2016-09-30,2016-09-29,2016-09-28,2016-09-27,2016-09-26,2016-09-25,2016-09-24,2016-09-23,2016-09-22,2016-09-21,2016-09-20,2016-09-19,2016-09-18,2016-09-17,2016-09-16,2016-09-15,2016-09-14,2016-09-13,2016-0

In [0]:
first_day = 2 # 2017-09-13 is a Wednesday
test_columns_code = ['w%d_d%d' % ((first_day+i) // 7, (first_day + i) % 7) for i in range(63)]
test = pd.DataFrame(index=train.index, columns=test_columns_code)
test.head()

,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,w9_d0,w9_d1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
test.fillna(0, inplace=True)

test['Page'] = all_page
test.sort_values(by='Page', inplace=True)
test.reset_index(drop=True, inplace=True)

In [0]:
test = test.merge(train_key, how='left', on='Page', copy=False)
test.head()

,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,w9_d0,w9_d1,Page,PageTitle,Site,AccessAgent
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,!vote_en.wikipedia.org_all-access_all-agents,!vote,en,all-access_all-agents
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,!vote_en.wikipedia.org_all-access_spider,!vote,en,all-access_spider
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,!vote_en.wikipedia.org_desktop_all-agents,!vote,en,desktop_all-agents
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_all-agents","""Awaken,_My_Love!""",en,all-access_all-agents
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_spider","""Awaken,_My_Love!""",en,all-access_spider


In [0]:
y_cols = test.columns[:63]
y_cols

Index(['w0_d2', 'w0_d3', 'w0_d4', 'w0_d5', 'w0_d6', 'w1_d0', 'w1_d1', 'w1_d2',
       'w1_d3', 'w1_d4', 'w1_d5', 'w1_d6', 'w2_d0', 'w2_d1', 'w2_d2', 'w2_d3',
       'w2_d4', 'w2_d5', 'w2_d6', 'w3_d0', 'w3_d1', 'w3_d2', 'w3_d3', 'w3_d4',
       'w3_d5', 'w3_d6', 'w4_d0', 'w4_d1', 'w4_d2', 'w4_d3', 'w4_d4', 'w4_d5',
       'w4_d6', 'w5_d0', 'w5_d1', 'w5_d2', 'w5_d3', 'w5_d4', 'w5_d5', 'w5_d6',
       'w6_d0', 'w6_d1', 'w6_d2', 'w6_d3', 'w6_d4', 'w6_d5', 'w6_d6', 'w7_d0',
       'w7_d1', 'w7_d2', 'w7_d3', 'w7_d4', 'w7_d5', 'w7_d6', 'w8_d0', 'w8_d1',
       'w8_d2', 'w8_d3', 'w8_d4', 'w8_d5', 'w8_d6', 'w9_d0', 'w9_d1'],
      dtype='object')

In [0]:
test = test.reset_index()
test.head()

,index,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,w9_d0,w9_d1,Page,PageTitle,Site,AccessAgent
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,!vote_en.wikipedia.org_all-access_all-agents,!vote,en,all-access_all-agents
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,!vote_en.wikipedia.org_all-access_spider,!vote,en,all-access_spider
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,!vote_en.wikipedia.org_desktop_all-agents,!vote,en,desktop_all-agents
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_all-agents","""Awaken,_My_Love!""",en,all-access_all-agents
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_spider","""Awaken,_My_Love!""",en,all-access_spider


In [0]:
train_cols = ['d_%d' % i for i in range(train_norm.shape[1])]
len(train_cols)

366

In [0]:
train_norm.columns = train_cols
# train_all_norm.columns = train_cols

In [0]:
sites = train_key.Site.unique()
sites

array(['zh', 'fr', 'en', 'co', 'ru', 'ww', 'de', 'ja', 'es'], dtype=object)

In [0]:
test_site = pd.factorize(test.Site)[0]
test['Site_label'] = test_site

In [0]:
accesses = train_key.AccessAgent.unique()
accesses

array(['all-access_spider', 'desktop_all-agents', 'mobile-web_all-agents',
       'all-access_all-agents'], dtype=object)

In [0]:
test_access = pd.factorize(test.AccessAgent)[0]
test['Access_label'] = test_access

In [0]:
test.shape

(145063, 70)

In [0]:
test0 = test.copy()

In [0]:
y_norm_cols = [c+'_norm' for c in y_cols]
y_pred_cols = [c+'_pred' for c in y_cols]

In [0]:
train_norm_diff = train_norm - train_norm.shift(-1, axis=1)
train_norm_diff.head()

train_norm_diff7 = train_norm - train_norm.shift(-7, axis=1)
train_norm_diff7.head()

,d_0,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,d_251,d_252,d_253,d_254,d_255,d_256,d_257,d_258,d_259,d_260,d_261,d_262,d_263,d_264,d_265,d_266,d_267,d_268,d_269,d_270,d_271,d_272,d_273,d_274,d_275,d_276,d_277,d_278,d_279,d_280,d_281,d_282,d_283,d_284,d_285,d_286,d_287,d_288,d_289,d_290,d_291,d_292,d_293,d_294,d_295,d_296,d_297,d_298,d_299,d_300,d_301,d_302,d_303,d_304,d_305,d_306,d_307,d_308,d_309,d_310,d_311,d_312,d_313,d_314,d_315,d_316,d_317,d_318,d_319,d_320,d_321,d_322,d_323,d_324,d_325,d_326,d_327,d_328,d_329,d_330,d_331,d_332,d_333,d_334,d_335,d_336,d_337,d_338,d_339,d_340,d_341,d_342,d_343,d_344,d_345,d_346,d_347,d_348,d_349,d_350,d_351,d_352,d_353,d_354,d_355,d_356,d_357,d_358,d_359,d_360,d_361,d_362,d_363,d_364,d_365
0,0.133531,0.470004,0.174353,0.298493,0.109199,-0.441833,-0.832909,-0.336472,0.259511,-0.174353,-0.160343,-0.836248,-0.331357,0.693147,0.490623,0.405465,0.040822,-0.417735,1.049822,0.890973,0.139762,-0.676340,0.182322,0.182322,0.769133,-0.669050,-1.077559,-0.300105,1.512588,-0.470004,-0.182322,-0.313658,-0.157629,0.759105,0.523248,-0.613104,0.470004,0.087011,0.485508,0.826679,-0.405465,-0.117783,0.087011,0.068993,0.200671,-0.117783,0.100084,-0.310155,-0.798508,-0.167054,0.000000,-0.054067,-0.405465,-0.100084,0.715620,0.393043,0.080043,-0.794930,-0.100084,0.462624,0.271934,0.257829,-0.036368,-0.080043,0.215111,-0.213574,-0.302281,-0.810930,-0.693147,0.113329,0.039221,0.083382,-0.175891,-0.196710,0.182322,0.348307,-0.652325,-0.470004,-0.298493,-0.092373,-0.278713,0.068993,-0.080043,0.503905,0.470004,-0.229574,0.307485,0.615185,-0.068993,-0.207639,0.071459,-0.392042,0.197826,-0.858662,-0.667829,0.510826,0.421213,-0.138150,0.615185,0.287682,0.339868,0.830348,-1.329136,0.154151,0.255933,0.356675,-0.377294,0.479573,-0.302281,1.446919,-0.328504,0.133531,-0.496437,0.336472,0.080043,-0.042560,-0.117783,0.083382,0.211309,0.000000,-0.824175,-0.606136,0.470004,-0.543615,-0.160343,-1.573506,-0.502092,0.952009,0.526093,-1.582409,0.342945,0.251314,0.910212,0.082238,-0.860201,0.167054,1.582409,0.095310,0.154151,-0.310155,0.847298,0.773190,-0.986495,-0.427444,0.051293,-0.635989,0.862224,-0.236389,0.182322,-0.758744,0.044452,-0.422857,0.481838,-0.744441,-0.581922,0.105361,1.891843,0.146604,-0.950192,0.000000,-0.530628,-0.028988,-0.287682,0.379490,-0.884202,1.098612,-0.133531,0.818310,0.223144,-0.287682,-0.836248,0.693147,0.174353,-0.080043,0.223144,-0.068993,0.421213,0.223144,0.000000,-0.356675,0.080043,0.000000,0.510826,0.211309,-0.154151,0.362906,-0.405465,0.287682,-0.535518,-0.200671,-0.257829,0.113329,-0.223144,-0.412845,0.325422,0.940983,0.200671,0.257829,0.820981,0.000000,1.329136,-0.430783,0

In [0]:
train_norm = train_norm.iloc[:,::-1]
train_norm_diff7m = train_norm - train_norm.rolling(window=7, axis=1).median()
train_norm = train_norm.iloc[:,::-1]
train_norm_diff7m = train_norm_diff7m.iloc[:,::-1]

In [0]:
# load allVisits information
allVisits = pd.read_csv("meta.csv")
allVisits = allVisits.sort_values(by='Page')
allVisits.head()

,Page,AllVisits
37206,!vote_en.wikipedia.org_all-access_all-agents,0.916291
32275,!vote_en.wikipedia.org_all-access_spider,0.405465
8357,!vote_en.wikipedia.org_desktop_all-agents,0.916291
37207,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_all-agents",0.000000
32276,"""Awaken,_My_Love!""_en.wikipedia.org_all-access_spider",0.000000


In [0]:
def add_median(test, train, train_diff, train_diff7, train_diff7m,
               train_key, periods, max_periods):
    train =  train.iloc[:,:7*max_periods]
    
    test = test.merge(allVisits, how='left', on='Page', copy=False)
    test.AllVisits = test.AllVisits.fillna(0).astype('float32')
    
    for site in sites:
        test[site] = (1 * (test.Site == site)).astype('float32')
    
    for access in accesses:
        test[access] = (1 * (test.AccessAgent == access)).astype('float32')

    for (w1, w2) in periods:
        
        df = train_key[['Page']].copy()
        
        c = 'median_%d_%d' % (w1, w2)
        cm = 'mean_%d_%d' % (w1, w2)
        cmax = 'max_%d_%d' % (w1, w2)
        
        cd = 'median_diff_%d_%d' % (w1, w2)
        cd7 = 'median_diff7_%d_%d' % (w1, w2)
        cd7m = 'median_diff7m_%d_%d' % (w1, w2)
        cd7mm = 'mean_diff7m_%d_%d' % (w1, w2)
        
        df[c] = train.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        df[cm] = train.iloc[:,7*w1:7*w2].mean(axis=1, skipna=True) 
        df[cmax] = train.iloc[:,7*w1:7*w2].max(axis=1, skipna=True) 
        df[cd] = train_diff.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        df[cd7] = train_diff7.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        df[cd7m] = train_diff7m.iloc[:,7*w1:7*w2].median(axis=1, skipna=True) 
        df[cd7mm] = train_diff7m.iloc[:,7*w1:7*w2].mean(axis=1, skipna=True) 
        
        test = test.merge(df, how='left', on='Page', copy=False)
        test[c] = (test[c] - test.AllVisits).fillna(0).astype('float32')
        test[cm] = (test[cm] - test.AllVisits).fillna(0).astype('float32')
        test[cmax] = (test[cmax] - test.AllVisits).fillna(0).astype('float32')
        test[cd] = (test[cd] ).fillna(0).astype('float32')
        test[cd7] = (test[cd7] ).fillna(0).astype('float32')
        test[cd7m] = (test[cd7m] ).fillna(0).astype('float32')
        test[cd7mm] = (test[cd7mm] ).fillna(0).astype('float32')

    for c_norm, c in zip(y_norm_cols, y_cols):
        test[c_norm] = (np.log1p(test[c]) - test.AllVisits).astype('float32')

    gc.collect()

    return test

max_periods = 52
periods = [(0,1), (1,2), (2,3), (3,4), 
           (4,5), (5,6), (6,7), (7,8),
           (0,2), (2,4),(4,6),(6,8),
           (0,4),(4,8),(8,12),(12,16),
           (0,8), (8,16), (0,12), (0,16),
#            (0, 20), (0, 24), (0, 36), (0, 52),
           (36, 52), (51, 52), (35, 36),
          ]

site_cols = list(sites)
access_cols = list(accesses)

for c in y_pred_cols:
    test[c] = np.NaN

test1 = add_median(test0, train_norm, train_norm_diff, train_norm_diff7, train_norm_diff7m, 
                   train_key, periods, max_periods)

test1.head()

,index,w0_d2,w0_d3,w0_d4,w0_d5,w0_d6,w1_d0,w1_d1,w1_d2,w1_d3,w1_d4,w1_d5,w1_d6,w2_d0,w2_d1,w2_d2,w2_d3,w2_d4,w2_d5,w2_d6,w3_d0,w3_d1,w3_d2,w3_d3,w3_d4,w3_d5,w3_d6,w4_d0,w4_d1,w4_d2,w4_d3,w4_d4,w4_d5,w4_d6,w5_d0,w5_d1,w5_d2,w5_d3,w5_d4,w5_d5,w5_d6,w6_d0,w6_d1,w6_d2,w6_d3,w6_d4,w6_d5,w6_d6,w7_d0,w7_d1,w7_d2,w7_d3,w7_d4,w7_d5,w7_d6,w8_d0,w8_d1,w8_d2,w8_d3,w8_d4,w8_d5,w8_d6,w9_d0,w9_d1,Page,PageTitle,Site,AccessAgent,Site_label,Access_label,AllVisits,zh,fr,en,co,ru,ww,de,ja,es,all-access_spider,desktop_all-agents,mobile-web_all-agents,all-access_all-agents,median_0_1,mean_0_1,max_0_1,median_diff_0_1,median_diff7_0_1,median_diff7m_0_1,mean_diff7m_0_1,median_1_2,mean_1_2,max_1_2,median_diff_1_2,median_diff7_1_2,median_diff7m_1_2,mean_diff7m_1_2,median_2_3,mean_2_3,max_2_3,median_diff_2_3,median_diff7_2_3,median_diff7m_2_3,mean_diff7m_2_3,median_3_4,mean_3_4,max_3_4,median_diff_3_4,median_diff7_3_4,median_diff7m_3_4,mean_diff7m_3_4,median_4_5,mean_4_5,max_4_5,median_diff_4_5,median_diff7_4_5,median_diff7m_4_5,mean_diff7m_4_5,median_5_6,mean_5_6,max_5_6,median_diff_5_6,median_diff7_5_6,median_diff7m_5_6,mean_diff7m_5_6,median_6_7,mean_6_7,max_6_7,median_diff_6_7,median_diff7_6_7,median_diff7m_6_7,mean_diff7m_6_7,median_7_8,mean_7_8,max_7_8,median_diff_7_8,median_diff7_7_8,median_diff7m_7_8,mean_diff7m_7_8,median_0_2,mean_0_2,max_0_2,median_diff_0_2,median_diff7_0_2,median_diff7m_0_2,mean_diff7m_0_2,median_2_4,mean_2_4,max_2_4,median_diff_2_4,median_diff7_2_4,median_diff7m_2_4,mean_diff7m_2_4,median_4_6,mean_4_6,max_4_6,median_diff_4_6,median_diff7_4_6,median_diff7m_4_6,mean_diff7m_4_6,median_6_8,mean_6_8,max_6_8,median_diff_6_8,median_diff7_6_8,median_diff7m_6_8,mean_diff7m_6_8,median_0_4,mean_0_4,max_0_4,median_diff_0_4,median_diff7_0_4,median_diff7m_0_4,mean_diff7m_0_4,median_4_8,mean_4_8,max_4_8,median_diff_4_8,median_diff7_4_8,median_diff7m_4_8,mean_diff7m_4_8,median_8_12,mean_8_12,max_8_12,median_diff_8_12,median_diff7_8_12,median_diff7m_8_12,mean_diff7m_8_12,median_12_16,mean_12_16,max_12_16,median_diff_12_16,median_diff7_12_16,median_diff7m_12_16,mean_diff7m_12_16,median_0_8,mean_0_8,max_0_8,median_diff_0_8,median_diff7_0_8,median_diff7m_0_8,mean_diff7m_0_8,median_8_16,mean_8_16,max_8_16,median_diff_8_16,median_diff7_8_16,median_diff7m_8_16,mean_diff7m_8_16,median_0_12,mean_0_12,max_0_12,median_diff_0_12,median_diff7_0_12,median_diff7m_0_12,mean_diff7m_0_12,median_0_16,mean_0_16,max_0_16,median_diff_0_16,median_diff7_0_16,median_diff7m_0_16,mean_diff7m_0_16,median_36_52,mean_36_52,max_36_52,median_diff_36_52,median_diff7_36_52,median_diff7m_36_52,mean_diff7m_36_52,median_51_52,mean_51_52,max_51_52,median_diff_51_52,median_diff7_51_52,median_diff7m_51_52,mean_diff7m_51_52,median_35_36,mean_35_36,max_35_36,median_diff_35_36,median_diff7_35_36,median_diff7m_35_36,mean_diff7m_35_36,w0_d2_norm,w0_d3_norm,w0_d4_norm,w0_d5_norm,w0_d6_norm,w1_d0_norm,w1_d1_norm,w1_d2_norm,w1_d3_norm,w1_d4_norm,w1_d5_norm,w1_d6_norm,w2_d0_norm,w2_d1_norm,w2_d2_norm,w2_d3_norm,w2_d4_norm,w2_d5_norm,w2_d6_norm,w3_d0_norm,w3_d1_norm,w3_d2_norm,w3_d3_norm,w3_d4_norm,w3_d5_norm,w3_d6_norm,w4_d0_norm,w4_d1_norm,w4_d2_norm,w4_d3_norm,w4_d4_norm,w4_d5_norm,w4_d6_norm,w5_d0_norm,w5_d1_norm,w5_d2_norm,w5_d3_norm,w5_d4_norm,w5_d5_norm,w5_d6_norm,w6_d0_norm,w6_d1_norm,w6_d2_norm,w6_d3_norm,w6_d4_norm,w6_d5_norm,w6_d6_norm,w7_d0_norm,w7_d1_norm,w7_d2_norm,w7_d3_norm,w7_d4_norm,w7_d5_norm,w7_d6_norm,w8_d0_norm,w8_d1_norm,w8_d2_norm,w8_d3_norm,w8_d4_norm,w8_d5_norm,w8_d6_norm,w9_d0_norm,w9_d1_norm
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,!vote_en.wikipedia.org_all-access_all-agents,!vote,en,all-access_all-agents,0,0,0.916291,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.693147,0.722165,1.223775,-0.262364,-0.262364,-0.223144,-0.135516,1.029619,1.031658,1.386294,-0.048790,0.000000,-0.074108,-0.152975,1.280934,1.114166,1.609438,0.251314,0.117783,0.241162,0.186451,0.788457,0.8

## Models

### Release memory before training models

In [0]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))
del train_all, train_norm, train_norm_diff7m
del orig
gc.collect()

                          orig: 908.0MiB
                     train_all: 888.7MiB
             train_norm_diff7m: 405.1MiB
                         test1: 256.1MiB
                         train: 202.5MiB
                    train_norm: 202.5MiB
               train_norm_diff: 202.5MiB
              train_norm_diff7: 202.5MiB
                          test: 193.0MiB
                         test0: 123.3MiB


13898

## Generate output for Kaggle

### Prediction

In [0]:
num_cols = (['median_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['mean_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['max_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['median_diff_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['median_diff7m_%d_%d' % (w1,w2) for (w1,w2) in periods])
num_cols.extend(['mean_diff7m_%d_%d' % (w1,w2) for (w1,w2) in periods])

test2 = test1
Xm, Xs, Xa, y = test2[num_cols].values, test2[site_cols].values, test2[access_cols].values, test2[y_norm_cols].values

In [0]:
from keras.models import Model, load_model
model = load_model('DNN_model7.h5')

Xm_train, Xs_train, Xa_train, y_train = test2[num_cols].values, test2[site_cols].values, test2[access_cols].values, test2[y_norm_cols].values

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [0]:
predY = model.predict([Xm_train, Xs_train, Xa_train])
predY = predY + test2.AllVisits.values.reshape((-1,1))
predY = np.expm1(predY)
predY[predY < 0.5 * offset] = 0

predY = (predY/offset).round()
test.loc[:, y_pred_cols] = predY

### Loading test key

In [0]:
test_id = pd.read_csv('../data/key_2.csv')

test_id['Date'] = [page[-10:] for page in tqdm(test_id.Page)]
test_id['Page'] = [page[:-11] for page in tqdm(test_id.Page)]
test_id.head()

100%|██████████| 8993906/8993906 [00:05<00:00, 1674774.53it/s]


,Page,Id,Date
0,007_スペクター_ja.wikipedia.org_all-access_all-agents,0b293039387a,2017-09-13
1,007_スペクター_ja.wikipedia.org_all-access_all-agents,7114389dd824,2017-09-14
2,007_スペクター_ja.wikipedia.org_all-access_all-agents,057b02ff1f09,2017-09-15
3,007_スペクター_ja.wikipedia.org_all-access_all-agents,bd2aca21caa3,2017-09-16
4,007_スペクター_ja.wikipedia.org_all-access_all-agents,c0effb42cdd5,2017-09-17


In [0]:
test_all = test_id.drop('Id', axis=1)
test_all['Visits_true'] = np.NaN
test_all = test_all.pivot(index='Page', columns='Date', values='Visits_true').astype('float32').reset_index()

# test_all.sort_values(by='Page', inplace=True)
test_all.reset_index(drop=True, inplace=True)

test_columns_date = list(test_all.columns[1:])

In [0]:
filename = 'wiki_traffic'

test_columns_save = [c+'_pred' for c in test_columns_code[1:]]
test_columns_save.append('Page')
test_save = test[test_columns_save]
test_save.columns = test_columns_date + ['Page']
test_save = pd.melt(test_save, id_vars=['Page'], var_name='Date', value_name='Visits')

test_sub = test_id.merge(test_save, how='left', on=['Page','Date'])

# test_sub_sorted = test_sub[['Id', 'Visits']].sort_values(by='Id')
# test_sub_sorted[['Id', 'Visits']].to_csv('../submissions/%s_test_sorted.csv' % filename, index=False)
test_sub[['Id', 'Visits']].to_csv('../submissions/%s_te.csv' % filename, index=False)